In [33]:
from sage.graphs.graph_decompositions.tree_decomposition import *
from sage.graphs.generators.basic import CompleteGraph
import time
import cProfile

In [34]:
def generate_graph_pathwidth_two(vertices):
    k = 3
    g = CompleteGraph(k)
    while k <= vertices-1:
        g.add_vertex(k)
        g.add_edge(k, k-1)
        g.add_edge(k, k-2)
        k += 1
    #show(g)
    return g

In [35]:
def compute_mis(g, td):
    start = time.time()
    
    def powerset(s):
        x = len(s)
        masks = [1 << i for i in range(x)]
        for i in range(1 << x):
            yield [ss for mask, ss in zip(masks, s) if i & mask]
            
    max_nei = 0
    for v in td.vertices():
        if len(td.neighbors(v)) > max_nei:
            max_nei = len(td.neighbors(v))
            root_node = v
            
    bags = {}
    q = [(root_node, 0)]
    visit = set()
    leaf_nodes = []
    while q:
        childPresent = False
        node, level = q.pop(0)
        visit.add(node)
        if level not in bags:
            bags[level] = []
        bags[level].append(node)
        for nei in td.neighbors(node):
            if nei not in visit:
                childPresent = True
                q.append((nei, level+1))
        if childPresent == False:
            leaf_nodes.append(node)

    bag_number = {}
    bn = 0
    for k, ver in bags.items():
        for v in ver:
            bag_number[v] = bn
            bn += 1

    number_to_bag = {}
    for k, v in bag_number.items():
        number_to_bag[v] = k

    # Initialize table A
    a_c = len(td.vertices())
    A = {None:{}}
    
    bags_processed_A = [False for _ in range(a_c)]
    for b in leaf_nodes:
        for v in b:
            if v not in A:
                A[v] = {}
            A[v][bag_number[b]] = {v}
        bags_processed_A[bag_number[b]] = True

    leaf_pairs = []
    q = [root_node]
    visit = set()
    while q:
        node = q.pop(0)
        visit.add(node)
        for nei in td.neighbors(node):
            if nei not in visit:
                q.append(nei)
                if node in leaf_nodes or nei in leaf_nodes:
                    leaf_pairs.append((bag_number[nei], bag_number[node]))

    B = {None:{}}
    # Bottom of B
    for i, j in leaf_pairs:
        # Fill None
        B[None][(i,j)] = number_to_bag[i] - number_to_bag[j]
        
        # Fill Leaf Pairs
        inter = number_to_bag[i] & number_to_bag[j]
        for v in inter:
            if v not in B:
                B[v] = {}
            if v in (number_to_bag[i] & number_to_bag[j]):
                B[v][(i,j)] = {v}

    for i in range(len(bags_processed_A)-1, -1, -1):
        if bags_processed_A[i]:
            continue
            
        # Compute A(S, i)
        powerset_of_bag_node = powerset(number_to_bag[i])
        for k in powerset_of_bag_node:
            tmp = set(k)
            for nei in td.neighbors(number_to_bag[i]):
                j = bag_number[nei]
                if j < i:
                    continue
                common = set(k) & set(nei)
                if len(common) == 0:
                    if (j, i) in B[None]:
                        tmp = tmp | set(B[None][(j, i)])
                elif len(common) == 1:
                    v = common.pop()
                    if v in B and (j, i) in B[v]:
                        tmp = tmp | (B[v][(j, i)] - (common & set(number_to_bag[j])))
                else:
                    v = tuple(common)
                    if v in B and (j, i) in B[v]:
                        tmp = tmp | (B[v][(j, i)] - (common & set(number_to_bag[j])))
            if len(k) == 0:
                s = None
            elif len(k) == 1:
                s = k[0]
            else:
                s = tuple(k)    
                if s not in B:
                    continue
            if s not in A:
                A[s] = {}
            A[s][i] = tmp
            if len(tmp) == 1:
                temp = tmp.pop()
            elif len(tmp) > 1:
                temp = tuple(tmp)
            if temp not in A:
                A[temp] = {}
                A[temp][i] = tmp

        # Compute B(S, i, j)
        for nei in td.neighbors(number_to_bag[i]):
            j = bag_number[nei]
            if j > i:
                continue

            for l in powerset(number_to_bag[i]):
                if len(l) == 0:
                    s_prime = None
                elif len(l) == 1:
                    s_prime = l[0]
                else:
                    s_prime = tuple(l)
                k = set(l) & set(number_to_bag[j])
                if len(k) == 0:
                    s = None
                elif len(k) == 1:
                    s = k.pop()
                else:
                    s = tuple(k)
                if (s_prime not in A) or (i not in A[s_prime]):
                    continue
                    
                if s not in B:
                    B[s] = {}
                if (i, j) not in B[s]:
                    B[s][(i, j)] = A[s_prime][i]
                else:
                    if len(B[s][(i, j)]) <= len(A[s_prime][i]):
                        B[s][(i, j)] = A[s_prime][i]
                        
        bags_processed_A[i] = True
        
    mis_set = {}
    for v in A.keys():
        if 0 in A[v] and len(A[v][0]) > len(mis_set):
            mis_set = A[v][0]
    
    end = time.time()
    print("Time Taken:", end-start)
    
    return (mis_set, len(mis_set))

In [36]:
g = generate_graph_pathwidth_two(1000)
td = g.treewidth(certificate=True)

In [37]:
cProfile.run('compute_mis(g, td)')

Time Taken: 0.9196884632110596
         558214 function calls (556223 primitive calls) in 0.929 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.279    0.279    0.920    0.920 298365473.py:1(compute_mis)
    17919    0.132    0.000    0.248    0.000 298365473.py:4(powerset)
        1    0.000    0.000    0.000    0.000 298365473.py:49(<listcomp>)
     1991    0.003    0.000    0.003    0.000 298365473.py:6(<listcomp>)
    15928    0.037    0.000    0.037    0.000 298365473.py:8(<listcomp>)
        1    0.009    0.009    0.929    0.929 <string>:1(<module>)
        8    0.000    0.000    0.000    0.000 category.py:1827(or_subcategory)
        8    0.000    0.000    0.000    0.000 category_with_axiom.py:1973(__classcall__)
        8    0.000    0.000    0.000    0.000 dynamic_class.py:129(dynamic_class)
     1998    0.000    0.000    0.000    0.000 generic_graph.py:11215(vertex_iterator)
    35873    0.087    0.0